In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

import matplotlib.pyplot as plt
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

In [2]:
config = {'lr': 0.00034439316653688684,
 'layers': 3,
 'step_size': 11,
 'gamma': 0.761795969995615,
 'bptt': 19,
 'dropout': 0.1227497445640586}

In [3]:
config['lr'] = config['lr'] * config['gamma'] **5
config['lr']

8.835840748963752e-05

In [4]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [5]:
dataset = pd.read_csv('/home/urwa/Documents/side_projects/urban/data/featureData/jfk.csv')

In [6]:
dataset.shape

(8757, 1049)

In [7]:
dataset.head(3)

,Date,Hour,1,10,100,101,102,106,107,108,...,91_lag_3,92_lag_3,93_lag_3,94_lag_3,95_lag_3,96_lag_3,97_lag_3,98_lag_3,99_lag_3,arrival_lag_3
0,2018-01-01,3,0,0,0,0,0,0,0,0,...,1.0,1.0,0.0,1.0,6.0,0.0,1.0,0.0,0.0,6.0
1,2018-01-01,4,0,3,0,0,1,0,0,1,...,4.0,1.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,6.0
2,2018-01-01,5,0,4,0,0,1,2,3,1,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,2.0


In [8]:
lag_columns = [c for c in dataset.columns if 'lag' in c]
len(lag_columns)

777

In [9]:
dataset = dataset[[c for c in dataset.columns if c not in lag_columns]]
dataset.shape

(8757, 272)

In [10]:
DateColumns = ['Date']

ext_columns = ['Dow', 'arrival','maxtemp', 'mintemp', 'avgtemp', 'departure', 'hdd',
       'cdd', 'participation', 'newsnow', 'snowdepth', 'ifSnow']

targetColumns = [c for c in dataset.columns if c not in ext_columns and \
                c not in DateColumns and c not in lag_columns and c != 'Hour']
len(targetColumns)

258

In [11]:
features_cols = [c for c in dataset.columns if c not in targetColumns and c not in DateColumns]
len(features_cols)

13

In [12]:
def create_inout_sequences(x,y, tw):
    inout_seq = []
    L = len(x)
    for i in range(L-tw):
        train_seq_x = x[i:i+tw]
        train_seq_y = y[i:i+tw]
#         train_seq = torch.cat((train_seq_x,train_seq_y),axis=1)
        
#         train_label = y[i+tw:i+tw+1]
        train_label = y[i+1:i+tw+1]
        inout_seq.append((train_seq_x, train_seq_y ,train_label))
    return inout_seq

In [13]:
class LSTM(nn.Module):
    def __init__(self, input_size=1, hidden_size=100, num_layers=1, dropout=0):
        super().__init__()
                
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        
        self.hidden_cell = (torch.zeros(num_layers,1,self.hidden_size).to(device),
                    torch.zeros(num_layers,1,self.hidden_size).to(device))
        
        self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size, num_layers=num_layers, dropout=dropout)


    def forward(self, x):
        self.hidden_cell = (torch.zeros(self.num_layers,1,self.hidden_size).to(device),
                    torch.zeros(self.num_layers,1,self.hidden_size).to(device))
           
        lstm_out, self.hidden_cell = self.lstm(x.view(len(x) ,1, -1), self.hidden_cell)
        
        return lstm_out, self.hidden_cell

In [14]:
class GraphPrediction(nn.Module):
    def __init__(self, feat_size=1, hidden_layer_size=100, network_size=1, layers=1, communities=10, ensembles=1, dropout=0, at_mat=None):
        super().__init__()
        
        # aggregation
        if at_mat != None:
            self.attachment_matrix = torch.nn.Parameter(at_mat)
            self.attachment_matrix.requires_grad = False
        else:
            self.attachment_matrix = torch.nn.Parameter(torch.randn(network_size,communities))
            self.attachment_matrix.requires_grad = True
        
        lstm_input = communities + feat_size
        
        self.ensembles = ensembles
        self.lstms = nn.ModuleList()
        self.linears = nn.ModuleList()
        for i in range(ensembles):
             self.lstms.append(LSTM(input_size=lstm_input, hidden_size=hidden_layer_size, num_layers=layers))
             self.linears.append(nn.Linear(hidden_layer_size, network_size))
            

    def forward(self, input_seq, feat):
        
        w = F.softmax(self.attachment_matrix, dim=1)
        x = torch.matmul(input_seq, self.attachment_matrix)
        x = torch.cat((x,feat),axis=1)
        x = x.view(len(input_seq) ,1, -1)
        
        predictions = []
        for i in range(self.ensembles):
            if torch.randn(1) < 0.7 or i==0 or not self.training:
                lstm_out, self.hidden_cell = self.lstms[i](x)
                y = self.linears[i](lstm_out.view(len(input_seq), -1))
                predictions.append(y)
        
        predictions = torch.stack(predictions)
#         print(predictions.shape)
        return predictions

In [15]:
def evaluate(model):
    model.eval()
    prediction = []
    with torch.no_grad():
        for feat,seq, labels in test_inout_seq:
#             model.hidden = (torch.zeros(layers, 1, model.hidden_layer_size),
#                             torch.zeros(layers, 1, model.hidden_layer_size))
            y = model(seq,feat).mean(dim=0)[-1]
    
            prediction.append(y)

    y_test_ = torch.stack([labels[-1] for feat,seq, labels in test_inout_seq], axis=0).detach().cpu().numpy()
    y_pred_ = torch.stack(prediction).detach().cpu().numpy()

    res = y_pred_ - y_test_
    r2 = r2_score(y_test_, y_pred_, multioutput='variance_weighted')
    rmse = mean_squared_error(y_test_, y_pred_)
    mae = mean_absolute_error(y_test_, y_pred_)
#     print("r2: ",r2)
    return (res, r2, rmse, mae)

In [16]:
def get_at_mat(targetColumns):
    comms = pd.read_csv('/home/urwa/Documents/side_projects/urban/UrbanTemporalNetworks/Data/ZonetoComm.csv')  
    communities = list(set(comms.start_community))

    mapping = dict(zip(comms.start_id, comms.start_community))
    comm_to_index = dict(zip(communities,range(len(communities))))
    col_to_index = dict(zip(targetColumns,range(len(targetColumns))))

    attach = torch.zeros(len(targetColumns), len(communities))

    for t_c in targetColumns:
        com = mapping[int(t_c)]
        x_i = col_to_index[t_c]
        y_i = comm_to_index[com]

        attach[x_i,y_i] = 1

    return attach

In [17]:
pretrained_weights = '/home/urwa/Documents/side_projects/urban/urban_traffic_prediciton/ensemble/bayesian_opt/jfk.pt'

In [18]:
bptt = config['bptt']

R2List = []
residual_list = []

for m in range(1,13):
    month_index  = pd.to_datetime(dataset.Date).dt.month == m
    
    print('-------------------------------------------------')
    print('-------------------------------------------------')
    print("Month: ", m)


    trainData = dataset[~month_index]
    testData = dataset[month_index]

    print("\n train test split")
    print(trainData.shape)
    print(testData.shape)


    print("\n ")
    X_train = trainData[features_cols].values
    X_train = torch.tensor(X_train).float().to(device)
    print(X_train.shape)

    y_train = trainData[targetColumns].values
    y_train = torch.tensor(y_train).float().to(device)
    print(y_train.shape)

    X_test = testData[features_cols].values
    X_test = torch.tensor(X_test).float().to(device)
    print(X_test.shape)

    y_test = testData[targetColumns].values
    y_test = torch.tensor(y_test).float().to(device)
    print(y_test.shape)
    
    
    train_inout_seq = create_inout_sequences(X_train,y_train, bptt)
    
    test_inout_seq = create_inout_sequences(X_test,y_test, bptt)
    print("\n sequences")
    print(train_inout_seq[0][0].shape,train_inout_seq[0][1].shape, train_inout_seq[0][2].shape)
    
    at_mat = get_at_mat(targetColumns)
    print("\nattachment matrix")
    print(at_mat.shape)
    
    
    layers = config['layers']
    communities = 24
    network_size = len(targetColumns)
    feat_size = len(features_cols)
    ensembles=10
    dropout = config['dropout']

    model = GraphPrediction(feat_size = feat_size, hidden_layer_size=100,
                 network_size=network_size, layers=layers,
                communities=communities, ensembles=ensembles, dropout=dropout, at_mat=at_mat).to(device)

    loss_function = nn.L1Loss()   
    optimizer = torch.optim.Adam(model.parameters(), lr=config['lr'])
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=config['step_size'], gamma=config['gamma'])
    print("\n model inititalized")
    
    model.load_state_dict(torch.load(pretrained_weights))
    print("\n model loaded")
    
    residual, r2, rmse, mae = evaluate(model)
    print("Pretraining R2: ",r2)
    
    
    best_r2 = 0
    best_residual = residual
#     torch.save(model.state_dict(), 'data/'+'jfk_'+str(m)+'.pt')
#     np.save('data/'+'jfk_'+str(m)+'.npy', best_residual)
    
    
    epochs = 15

    for i in range(epochs):
        model.train()
        for feat,seq, labels in train_inout_seq:
            optimizer.zero_grad()
    #         model.hidden_cell = (torch.zeros(layers, 1, model.hidden_layer_size).to(device),
    #                         torch.zeros(layers, 1, model.hidden_layer_size).to(device))

            y_pred = model(seq, feat)
            labels = labels.repeat(y_pred.shape[0],1,1)

            single_loss = loss_function(y_pred, labels)
            single_loss.backward()
            optimizer.step()

        scheduler.step()
        residual, r2, rmse, mae = evaluate(model)
        print(f'epoch: {i:3} loss: {single_loss.item():10.8f} r2: {r2:5.3f} rmse: {rmse:5.3f} mae: {mae:5.3f}')
        
        if r2 > best_r2:
            best_r2 = r2
            best_residual = residual
            torch.save(model.state_dict(), 'data/'+'jfk_'+str(m)+'.pt')
            np.save('data/'+'jfk_'+str(m)+'.npy', best_residual)


    print(f'epoch: {i:3} loss: {single_loss.item():10.10f}')
    print("bet_r2: ", best_r2)

    R2List.append(best_r2)
    residual_list.append(best_residual)   

-------------------------------------------------
-------------------------------------------------
Month:  1

 train test split
(8016, 272)
(741, 272)

 
torch.Size([8016, 13])
torch.Size([8016, 258])
torch.Size([741, 13])
torch.Size([741, 258])

 sequences
torch.Size([19, 13]) torch.Size([19, 258]) torch.Size([19, 258])

attachment matrix
torch.Size([258, 24])

 model inititalized

 model loaded
Pretraining R2:  0.7204843593885372
epoch:   0 loss: 1.34565318 r2: 0.614 rmse: 2.758 mae: 0.926
epoch:   1 loss: 1.34231913 r2: 0.620 rmse: 2.714 mae: 0.919
epoch:   2 loss: 1.29094350 r2: 0.622 rmse: 2.704 mae: 0.917
epoch:   3 loss: 1.27928150 r2: 0.623 rmse: 2.698 mae: 0.917
epoch:   4 loss: 1.26050305 r2: 0.623 rmse: 2.693 mae: 0.915
epoch:   5 loss: 1.25294054 r2: 0.623 rmse: 2.693 mae: 0.915
epoch:   6 loss: 1.23628235 r2: 0.624 rmse: 2.688 mae: 0.915
epoch:   7 loss: 1.22186852 r2: 0.625 rmse: 2.684 mae: 0.913
epoch:   8 loss: 1.22072709 r2: 0.625 rmse: 2.682 mae: 0.913
epoch:   9 los

epoch:  12 loss: 1.20413482 r2: 0.634 rmse: 2.682 mae: 0.941
epoch:  13 loss: 1.19397664 r2: 0.635 rmse: 2.676 mae: 0.940
epoch:  14 loss: 1.19616759 r2: 0.635 rmse: 2.676 mae: 0.940
epoch:  14 loss: 1.1961675882
bet_r2:  0.6350797965169155
-------------------------------------------------
-------------------------------------------------
Month:  7

 train test split
(8013, 272)
(744, 272)

 
torch.Size([8013, 13])
torch.Size([8013, 258])
torch.Size([744, 13])
torch.Size([744, 258])

 sequences
torch.Size([19, 13]) torch.Size([19, 258]) torch.Size([19, 258])

attachment matrix
torch.Size([258, 24])

 model inititalized

 model loaded
Pretraining R2:  0.6892882980441339
epoch:   0 loss: 1.34333026 r2: 0.609 rmse: 2.751 mae: 0.958
epoch:   1 loss: 1.31705678 r2: 0.610 rmse: 2.747 mae: 0.959
epoch:   2 loss: 1.29852366 r2: 0.611 rmse: 2.737 mae: 0.958
epoch:   3 loss: 1.27320206 r2: 0.611 rmse: 2.740 mae: 0.959
epoch:   4 loss: 1.26234078 r2: 0.611 rmse: 2.740 mae: 0.959
epoch:   5 loss: 

epoch:   8 loss: 1.16690660 r2: 0.539 rmse: 3.374 mae: 1.013
epoch:   9 loss: 1.15782464 r2: 0.539 rmse: 3.376 mae: 1.014
epoch:  10 loss: 1.15421462 r2: 0.539 rmse: 3.375 mae: 1.014
epoch:  11 loss: 1.15184188 r2: 0.538 rmse: 3.383 mae: 1.013
epoch:  12 loss: 1.14866006 r2: 0.537 rmse: 3.389 mae: 1.014
epoch:  13 loss: 1.14592636 r2: 0.537 rmse: 3.388 mae: 1.014
epoch:  14 loss: 1.14623535 r2: 0.537 rmse: 3.387 mae: 1.014
epoch:  14 loss: 1.1462353468
bet_r2:  0.5414843036210735


In [19]:
R2List

[0.6292653363941075,
 0.6220370443486188,
 0.6576641046354088,
 0.652311024409751,
 0.6635955012709063,
 0.6350797965169155,
 0.6134558738917163,
 0.5773014550689083,
 0.6660435244138342,
 0.6003052940239731,
 0.5640616047920537,
 0.5414843036210735]

In [ ]:
0.6292653363941075
0.6220370443486188
0.6576641046354088
0.652311024409751
0.6635955012709063
0.6350797965169155
0.6134558738917163
0.5773014550689083
0.6660435244138342

In [20]:
len(set(attachment))

NameError: name 'attachment' is not defined

In [ ]:
# 2 linear
# 0.47

In [ ]:
# 1 linear
# 0.51

In [ ]:
# 1 linear + RELU
# 0.50

In [ ]:
# 1 linear bptt = 24
# 0.52

In [ ]:
#ensemble
#0.53228

In [ ]:
#  TODO
# other hubs
# ensemble model